##### [Modelo de extração de dados do Portal da Transparência de Sergipe](https://www.transparencia.se.gov.br/Pessoal/PorOrgao.xhtml)

##### Possibilidades de obtenção de dados:
- Por Orgão;
- Por Entidade.

> [Documentação aqui](https://serigy.tce.se.gov.br/comum/recursos-humanos.html#_6-1-folha-de-pagamento)


27-/07 - Diretoria de Modernização e Tecnologia (DMT) - (79) 3216-4433 - 4625

Valmor: Serigy é uma documentação que apresenta o padrão de codificação a ser implementados pelos municípios, e que permite ao crawler do TCESE obter os dadados e consolida-los. 


##### Importa bibliotecas

In [9]:
import pandas as pd

1. Cria pastas no diretório download

In [10]:
import os

# Caminho do diretório de downloads
download_dir = r"C:\Estudos FGV Projetos\2025-projeto-sergipe\base_dados_sergipe\downloads"
os.makedirs(download_dir, exist_ok=True)

# Cria pastas
folder_names = [
    "FES-27-EFETIVO",
    "FES-265-PSS",
    "FES-151-SAMU",
    "FHS-206-CLT-24910",
    "FHS-205-CLT-24900",
    "FHS-152-EFETIVO",
    "FHS-226-ESTAGIARIO",
    "FSPH-167-EFETIVO",
    "FUNESA-168",
    "RAW"
]

# executa a criação das pastas.
## se elas já existem, ignora. Uma vez que podem conter arquivos.
for folder_name in folder_names:
    folder_path = os.path.join(download_dir, folder_name)
    os.makedirs(folder_path, exist_ok=True)

2. Função de raspagem de dados

In [11]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import Select
from tqdm import tqdm

# Configurações do Chrome
chrome_options = Options()

#✅ Executa em segundo plano (sem abrir janela do navegador)
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

chrome_options.add_experimental_option("prefs", {
    "download.default_directory": r"C:\Estudos FGV Projetos\2025-projeto-sergipe\base_dados_sergipe\downloads\RAW",
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
    "profile.default_content_settings.popups": 0,
    "profile.default_content_setting_values.automatic_downloads": 1,
})

# ✅ Inicia o driver com as opções
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 20)

# Função para configurar os filtros e baixar o CSV
def setaConfigs(wait, driver, mes, ano, orgao):

    resultado = f"{orgao}-{mes}-{ano}"

    # Sessão em branco
    driver.get("about:blank")
    time.sleep(2)

    # Chama o driver a cada iteração.
    try:
        driver.get("https://www.transparencia.se.gov.br/Pessoal/PorOrgao.xhtml?faces-redirect=true")
        time.sleep(5)
    except WebDriverException as e:
        print(f"Erro ao carregar a página: {e}")

    # Imprime o órgão e o período
    #print(f"{orgao} -> {mes}/{ano}")
    time.sleep(2)

    # Clica no filtro "Orgão"
    orgao_label = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='frmPrincipal:j_idt170:0']")))
    orgao_label.click()
    time.sleep(5)

    # Seleciona o ano
    select_ano = wait.until(EC.element_to_be_clickable((By.ID, "frmPrincipal:ano")))
    Select(select_ano).select_by_value(str(ano))
    time.sleep(2)

    # Seleciona o mês
    select_mes = wait.until(EC.element_to_be_clickable((By.ID, "frmPrincipal:mes")))
    Select(select_mes).select_by_value(str(mes))
    time.sleep(2)

    # Seleciona o órgão
    select_orgao = wait.until(EC.element_to_be_clickable((By.ID, "frmPrincipal:selOrgao")))
    select_element = Select(select_orgao)
    time.sleep(2)

    # Verifica se o valor está presente nas opções do dropdown
    options = [option.get_attribute("value") for option in select_element.options]
    if str(orgao) in options:
        select_element.select_by_value(str(orgao))
        time.sleep(5)

        # Clica em "Pesquisar"
        pesquisar = wait.until(EC.element_to_be_clickable((By.ID, "frmPrincipal:botaoPesquisar")))
        pesquisar.click()
        time.sleep(20)

        # 👉 Clica no botão de download .CSV
        wait.until(EC.invisibility_of_element((By.CLASS_NAME, "loadDiv")))

        # Clica no botão para baixar o CSV
        # Encontra o botão pelo seletor CSS e clica nele
        btn_csv = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button#frmPrincipal\\:j_idt197")))
        return btn_csv.click(), resultado, orgao
        
    # Limpar cookies e fechar o driver
    time.sleep(2)
    driver.delete_all_cookies()
    driver.quit()

3. Função aguardar download do arquivo

In [12]:
import os
import shutil
import time

# Função para aguardar o download de um arquivo CSV
def aguardar_csv(pasta, timeout=30):
    #print("⏳ Aguardando download do CSV...")
    tempo_inicio = time.time()
    while time.time() - tempo_inicio < timeout:
        arquivos_csv = [f for f in os.listdir(pasta) if f.endswith(".csv")]
        if arquivos_csv:
            return arquivos_csv[0]
        time.sleep(1)
    return None

4. Execução

In [24]:
# Parametros de execução
anos = range(2025, 2026)
meses = range(1, 7)
instituicoes = [27, 265, 151, 205, 206, 152, 226, 167, 168]

# as instituições analisadas são essas:
'''
"FES-27-EFETIVO",
"FES-265-PSS",
"FES-151-SAMU",
"FHS-206-CLT-24910",
"FHS-205-CLT-24900",
"FHS-152-EFETIVO",
"FHS-226-ESTAGIARIO",
"FSPH-167-EFETIVO",
"FUNESA-168"
'''

# Loop principal
for ano in anos:    
    for mes in tqdm(meses, desc="Meses", leave=False):
        for instituicao in tqdm(instituicoes, desc="Instituições", leave=False):
            print(f"Processando: {instituicao} - {mes}/{ano}")
            click_result, resultado, orgao = setaConfigs(wait, driver, mes, ano, instituicao)

            download_raw_dir = r"C:/Estudos FGV Projetos/2025-projeto-sergipe/base_dados_sergipe/downloads/RAW"
            download_base_dir = r"C:/Estudos FGV Projetos/2025-projeto-sergipe/base_dados_sergipe/downloads"

            orgao_str = str(orgao)
            pasta_alvo = next((folder for folder in folder_names if orgao_str in folder), None)

            if pasta_alvo:
                pasta_destino = os.path.join(download_base_dir, pasta_alvo)
                os.makedirs(pasta_destino, exist_ok=True)

                nome_arquivo = aguardar_csv(download_raw_dir, timeout=30)

                if nome_arquivo:
                    caminho_completo = os.path.join(download_raw_dir, nome_arquivo)

                    primeiraParte = nome_arquivo.replace('Folha-de-Pagamento-', '').split('-')[0]
                    segundaParte = nome_arquivo.replace('Folha-de-Pagamento-', '').split('-', 1)[1]
                    novo_nome = f"{primeiraParte}-{orgao}-{segundaParte}"

                    caminho_final = os.path.join(pasta_destino, novo_nome)
                    
                    # Mover o arquivo para a pasta de destino
                    shutil.move(caminho_completo, caminho_final)

                    # Verifica se o arquivo não está vazio antes de abrir
                    if os.path.getsize(caminho_final) > 0:
                        # Abrir o arquivo para leitura
                        with open(caminho_final, mode='r', encoding='utf-8') as arquivo_entrada:
                            dados = pd.read_csv(arquivo_entrada, sep=',', low_memory=False)
                    else:
                        print(f"⚠️ Arquivo vazio: {caminho_final}")
                        continue

                    # Modificar os dados
                    dados['mes'] = mes
                    dados['ano'] = ano
                    dados['num_orgao'] = orgao

                    # Abrir o arquivo para escrita (sobrescrevendo)
                    with open(caminho_final, mode='w', encoding='utf-8', newline='') as arquivo_saida:
                        dados.to_csv(arquivo_saida, index=False)
                else:
                    print("❌ CSV não foi encontrado ou não finalizou o download.")
            else:
                print(f"⚠️ Nenhuma pasta encontrada com o órgão {orgao}.")


Meses:   0%|          | 0/6 [00:00<?, ?it/s]

Processando: 27 - 1/2025


Processando: 265 - 1/2025


Processando: 151 - 1/2025


Processando: 205 - 1/2025


...

Processando: 206 - 6/2025


Processando: 152 - 6/2025


Processando: 226 - 6/2025


Processando: 167 - 6/2025


Processando: 168 - 6/2025


4.5 Problemas encontrados na coleta

Dados:

- Dados da FES-EFETIVO de julho de 2023 está indisponível no sistema;
- Dados da FES-PSS dos meses janeiro, fevereiro, março e abril de 2021 estão indisponíveis;
- Dados da FHS-CLT-24910 do mês de abril de 2025 não está diponível no sistema;
- FUNESA não mostra a lista completa de funcionários. A lista é limitada a um grupo de 5 a 7 pessoas. Os dados podem ser obtidos no [site da FUNESA](https://funesa.se.gov.br/transparencia/pessoal-2/);
- FSPH só disponibiliza dados dos funcionários efetivos. Não há dados para comissionados e estagiários. Esses dados só podem ser obtidos no [site da FSPH](https://www.hemose.se.gov.br/rh_transparencia.html).


5. Junção das bases

In [26]:
import os
import pandas as pd
import glob

download_base_dir = r"C:/Estudos FGV Projetos/2025-projeto-sergipe/base_dados_sergipe/downloads"

# Lista para armazenar os DataFrames válidos
lista_dfs = []

# Percorre todas as subpastas e arquivos CSV
for pasta_raiz, _, _ in os.walk(download_base_dir):
    arquivos_csv = glob.glob(os.path.join(pasta_raiz, "*.csv"))
    
    for caminho_csv in arquivos_csv:
        try:
            # Lê apenas as primeiras 5 linhas para verificar se há dados
            if os.path.getsize(caminho_csv) > 0:
                df = pd.read_csv(caminho_csv, sep=",", low_memory=False)

                # Ignora CSVs que existem mas estão tecnicamente vazios (sem colunas/dados)
                if not df.empty:
                    lista_dfs.append(df)
        except Exception as e:
            print(f"Erro ao processar {caminho_csv}: {e}")

# Concatena todos os DataFrames válidos
if lista_dfs:
    dados_concatenados = pd.concat(lista_dfs, ignore_index=True)
    print(f"Total de linhas unidas: {len(dados_concatenados)}")
    print(f"Total de arquivos processados: {len(lista_dfs)}")
    
    # (Opcional) Salvar em CSV final
    dados_concatenados.to_csv("C:/Estudos FGV Projetos/2025-projeto-sergipe/base_dados_sergipe/downloads/Consolidado/todos_os_dados_consolidados.csv", index=False)
else:
    print("Nenhum CSV válido encontrado.")


Total de linhas unidas: 1151546
Total de arquivos processados: 480


In [ ]:
## Para uma primeira aproximação ao tamanho da amostra
# n0 = tamanho incial da amostra
# E0 = erro amostral admissível (0,05 → 5%, 0,03 → 3%) elevado ao quadrado

# Fórmula = n0 = 1/(E0)^2
n0 = 1
E0 = 0.03
primeira_aprox = n0 / E0 ** 2
print(f"Tamanho da amostra para uma primeira estimativa: {round(primeira_aprox, 2)}")



In [ ]:
import math

Z = 1.96        # 95% de confiança
p = 0.5         # proporção segura
E = 0.03        # erro amostral de 3% (0.03)

n0 = (Z**2 * p * (1 - p)) / E**2
print(f"Tamanho da amostra infinita: {n0:.0f}")

In [ ]:
# Parâmetros
Z = 1.96      # 95% de confiança
p = 0.5       # proporção esperada
E = 0.03      # margem de erro (3%)
N = 1139047   # população finita

# Tamanho da amostra infinita
n0 = (Z**2 * p * (1 - p)) / (E**2)

# Correção para população finita
n = (N * n0) / (N + n0 - 1)

# Exibe o resultado
print(f"Tamanho da amostra (população finita): {round(n)}")